Deep learning is a great tool that helps us efficiently summarize inherent patterns from tons of input data. I'd like to introduce DeepLearning.scala by letting the framework learn the common difference from Arithmetic progression.

## Background

**Input**:
 Arithmetic progression(AP) as:
``` val input: INDArray = Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray``` 

**Output**: 
 Common Difference of the certain AP as: 
```val expectedOutput: INDArray = Array(Array(1), Array(3), Array(2)).toNDArray```

So here we want DeepLearning.scala to learn the common difference from the AP, i.e. ```{1} from {0, 1, 2} ``` 
in which `2-1 = 1-0 = 1 `

## Install DeepLearning.scala

DeepLearning.scala is hosted on Maven Central repository.

You can use magic imports in [jupyter-scala](https://github.com/alexarchambault/jupyter-scala) or [Ammonite-REPL](http://www.lihaoyi.com/Ammonite/#Ammonite-REPL) to download DeepLearning.scala and its dependencies.

In [12]:
import $ivy.`com.thoughtworks.deeplearning::jupyter-differentiable:2.0.0-M1`
interp.load.plugin.ivy("org.scalamacros" % "paradise_2.11.11" % "2.1.0")

import $ivy.`org.nd4j:nd4j-native-platform:0.7.2`

import $ivy.$                                                               

import $ivy.$                                    

If you use [sbt](http://www.scala-sbt.org), please add the following settings into your `build.sbt`:

``` scala
libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiable" % "latest.release"

addCompilerPlugin("org.scalamacros" % "paradise" % "2.1.0" cross CrossVersion.full)

fork := true

scalaVersion := "2.11.11"
```

Note that this example must run on Scala 2.11.11 because [nd4s](http://nd4j.org/scala) does not support Scala 2.12. Make sure there is not a setting like `scalaVersion := "2.12.x"` in your `build.sbt`.

See [Scaladex](https://index.scala-lang.org/thoughtworksinc/deeplearning.scala) to install DeepLearning.scala in other build tools!

Then, you may want to import classes in DeepLearning.scala and its dependencies.

In [13]:
import com.thoughtworks.deeplearning.math._
import com.thoughtworks.deeplearning.jupyter.differentiable.Any._
import com.thoughtworks.deeplearning.differentiable.INDArray.{
  Optimizer => INDArrayOptimizer
}
import INDArrayOptimizer.LearningRate
import com.thoughtworks.deeplearning.jupyter.differentiable.INDArray.implicits._
import com.thoughtworks.each.Monadic._
import com.thoughtworks.raii.asynchronous.Do
import com.thoughtworks.deeplearning.jupyter.differentiable.Double._
import com.thoughtworks.deeplearning.jupyter.differentiable.Double.implicits._
import com.thoughtworks.deeplearning.Tape
import com.thoughtworks.deeplearning.jupyter.differentiable
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4s.Implicits._
import scala.concurrent.ExecutionContext.Implicits.global
import scalaz.concurrent.Task
import scalaz.{-\/, \/, \/-}
import scalaz.std.vector._

import com.thoughtworks.deeplearning.math._

import com.thoughtworks.deeplearning.jupyter.differentiable.Any._

import com.thoughtworks.deeplearning.differentiable.INDArray.{
  Optimizer => INDArrayOptimizer
}

import INDArrayOptimizer.LearningRate

import com.thoughtworks.deeplearning.jupyter.differentiable.INDArray.implicits._

import com.thoughtworks.each.Monadic._

import com.thoughtworks.raii.asynchronous.Do

import com.thoughtworks.deeplearning.jupyter.differentiable.Double._

import com.thoughtworks.deeplearning.jupyter.differentiable.Double.implicits._

import com.thoughtworks.deeplearning.Tape

import com.thoughtworks.deeplearning.jupyter.differentiable

import org.nd4j.linalg.api.ndarray.INDArray

import org.nd4j.linalg.factory.Nd4j

import org.nd4s.Implicits._

import scala.concurrent.ExecutionContext.Implicits.global

import scalaz.concurrent.Task

import scalaz.{-\/, \/, \/-}

import scalaz.std.vector._

## Design your neural network

DeepLearning.scala is also a language that we can use to create complex neural networks.


In the following sections, you will learn:
 * how to create your neural network
 * how to train your neural network
 * how to predict your neural network

### Create your neural network

Same as the definition of a normal Scala function, the definition of neural network consists of a type definition for its parameter, a type definition for its return value, and a body that contains mathematical formulas, function-calls, and control flows.

#### Weight Intialization 

We will create a trainable neural network.
It means that some variables in the neural network can be changed automatically according to some goals. Those variables are called `weight`.
You can create weight variables via `toWeight` method, given its initial value.

In order to create a weight, you must create an `Optimizer`, which contains the rule that manages how the weight changes. 

In [14]:
implicit def optimizer: INDArrayOptimizer = new LearningRate {
  def currentLearningRate() = 0.001
}

defined function optimizer

In [15]:
val weight = (Nd4j.randn(3, 1) / scala.math.sqrt(3.0)).toWeight

weight: Do[differentiable.package.INDArray.INDArrayTape] = Suspend(<function0>)

#### define your neural network
Your neural network is just a normal scala function:

In [16]:
def myNeuralNetwork(input: INDArray): differentiable.INDArray = {
  dot(input, weight)
}

defined function myNeuralNetwork

## Train your Neural Network

You have learned that weight will be automatically changed due to some goals.

In DeepLearning.scala, when we train a neural network, our goal should always be minimizing the absolute of the return value.

For example, if someone repeatedly call `train(myNeuralNetwork(Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray))`,
the neural network would try to minimize `input dot weight`.
Soon `weight` would become an array of zeros in order to make `input dot weight` zeros,
and `predict(myNeuralNetwork(Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray))` would return `Array(Array(0), Array(0), Array(0)).toNDArray`.

What if you expect `predict(myNeuralNetwork(Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray))` to return `Array(Array(1), Array(3), Array(2)).toNDArray`?

You can create another neural network that evaluates how far between the result of `myNeuralNetwork` and your expectation. The new neural network is usually called **loss function**.

In [17]:
def lossFunction(input: INDArray,
                 expectOutput: INDArray): differentiable.Double = {
  sumT(abs(myNeuralNetwork(input) - expectOutput))
}

defined function lossFunction

When the `lossFunction` get trained continuously, its return value will be close to zero, and the result of  `myNeuralNetwork` must be close to the expected result at the same time.

Note the `lossFunction` accepts a `input` and `expectOutput` as its parameter.
The first array is the input data used to train the neural network, and the second array is the expected output.

Then we create a plot to show how the loss changed during iterations.

In [18]:
import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.2`

import $ivy.$                                             

In [19]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import plotly._

import plotly.element._

import plotly.layout._

import plotly.JupyterScala._

In [20]:
def polyLoss(lossSeq: IndexedSeq[Double]): Unit = {
  plotly.JupyterScala.init()

  val plot = Seq(
    Scatter(lossSeq.indices, lossSeq)
  )

  plot.plot(
    title = "loss by time"
  )
}

defined function polyLoss

Now, we hard-code some data to train the network:

In [21]:
val input: INDArray = Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray

val expectedOutput: INDArray = Array(Array(1), Array(3), Array(2)).toNDArray

@monadic[Task]
val trainTask: Task[Unit] = {

  val lossSeq = for (_ <- (1 to 400).toVector) yield {
    train(lossFunction(input, expectedOutput)).each
  }

  polyLoss(lossSeq)

}

input: INDArray = [[0.00, 1.00, 2.00],
 [3.00, 6.00, 9.00],
 [13.00, 15.00, 17.00]]
expectedOutput: INDArray = [1.00, 3.00, 2.00]
trainTask: Task[Unit] = scalaz.concurrent.Task@7670223f

`@monadic` and `throwableMonadic` is a syntax sugar provide by [each](https://github.com/ThoughtWorksInc/each).

After those iterations, the loss should close to zero.

## Predict  your Neural Network

In [22]:
val predictResult = throwableMonadic[Task] {
  trainTask.each
  predict(myNeuralNetwork(input)).each
}

predictResult.unsafePerformSyncAttempt match {
  case -\/(e) => throw e
  case \/-(result) =>
    println(result)
}

[1.01, 3.22, 2.87]


predictResult: Task[Tape.<refinement>.this.type.Data] = scalaz.concurrent.Task@235bbd14

## Summary

In this article, you have learned:
* to create neural networks dealing with complex data structures like `Double` and `INDArray` like ordinary programming language
* to train your neural network
* to predict your neural network